# 7. LangSmith を使った RAG アプリケーションの評価


In [ ]:
# google colab 用なのでコメントアウト
# import os
# from google.colab import userdata

# os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
# os.environ["LANGCHAIN_PROJECT"] = "agent-book"

## 7.4. Ragas による合成テストデータの生成


### パッケージのインストール


In [ ]:
# google colab 用なのでコメントアウト
# uv pyproject.toml で chap07以外用の dependencies をコメントアウトして、
# chap07用の dependencies をコメントインして、 `uv sync` して下さい。
# !pip install langchain-core==0.2.30 langchain-openai==0.1.21 \
#     langchain-community==0.2.12 GitPython==3.1.43 \
#     langchain-chroma==0.1.2 chromadb==0.5.3 \
#     ragas==0.1.14 nest-asyncio==1.6.0

In [ ]:
from dotenv import load_dotenv

# 事前に .env ファイルを作って、OPENAI_API_KEY, LANGCHAIN_ENDPOINT, などを設定してください
load_dotenv()

True

### 検索対象のドキュメントのロード


In [2]:
from langchain_community.document_loaders import GitLoader


def file_filter(file_path: str) -> bool:
    return file_path.endswith(".mdx")


loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="master",
    file_filter=file_filter,
)

documents = loader.load()
print(len(documents))

370


### Ragas による合成テストデータ生成の実装


In [3]:
for document in documents:
    document.metadata["filename"] = document.metadata["source"]

In [5]:
# filename と言いつつ、checkout した langchain リポジトリの top directory からの相対パスが入る.
# e.g., docs/docs/integrations/retrievers/self_query/index.mdx
# file_name には、ファイル名: index.mdx などが入っている
document.metadata

{'source': 'docs/docs/integrations/retrievers/self_query/index.mdx',
 'file_path': 'docs/docs/integrations/retrievers/self_query/index.mdx',
 'file_name': 'index.mdx',
 'file_type': '.mdx',
 'filename': 'docs/docs/integrations/retrievers/self_query/index.mdx'}

In [6]:
import nest_asyncio
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

nest_asyncio.apply()

generator = TestsetGenerator.from_langchain(
    generator_llm=ChatOpenAI(model="gpt-4o-mini"),  # きちんと評価したい場合は、gpt-4o, o1-mini, o1-preview を推奨
    critic_llm=ChatOpenAI(model="gpt-4o-mini"),  # きちんと評価したい場合は、gpt-4o, o1-mini, o1-preview を推奨
    embeddings=OpenAIEmbeddings(),
)

testset = generator.generate_with_langchain_docs(
    documents,
    test_size=4,
    distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25},
)


embedding nodes:   0%|          | 0/1150 [00:00<?, ?it/s]

Generating:   0%|          | 0/4 [00:00<?, ?it/s]

In [10]:
testset.to_pandas()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What is CerebriumAI and what services does it ...,[# CerebriumAI\n\n>[Cerebrium](https://docs.ce...,CerebriumAI is a serverless GPU infrastructure...,simple,[{'source': 'docs/docs/integrations/providers/...,True
1,What is the purpose of LLM observability in th...,[# PromptLayer\n\n>[PromptLayer](https://docs....,The purpose of LLM observability in the contex...,simple,[{'source': 'docs/docs/integrations/providers/...,True
2,What are the perks of AI customizing enterpris...,[# MindsDB\n\nMindsDB is the platform for cust...,The perks of AI customizing enterprise data in...,reasoning,[{'source': 'docs/docs/integrations/providers/...,True
3,What AI features does MindsDB provide with Mot...,[# MindsDB\n\nMindsDB is the platform for cust...,The answer to given question is not present in...,multi_context,[{'source': 'docs/docs/integrations/providers/...,True


In [12]:
# testset の中身を眺めるならどうぞ
testset_df = testset.to_pandas()
testset_df.to_csv("testset.csv", index=False, encoding="utf-8")
testset_df.to_json("testset.json", orient="records", force_ascii=False, indent=4)

### LangSmith の Dataset の作成


In [13]:
from langsmith import Client

dataset_name = "agent-book"

client = Client()

if client.has_dataset(dataset_name=dataset_name):
    client.delete_dataset(dataset_name=dataset_name)

dataset = client.create_dataset(dataset_name=dataset_name)

### 合成テストデータの保存


In [14]:
inputs = []
outputs = []
metadatas = []

for testset_record in testset.test_data:
    inputs.append(
        {
            "question": testset_record.question,
        }
    )
    outputs.append(
        {
            "contexts": testset_record.contexts,
            "ground_truth": testset_record.ground_truth,
        }
    )
    metadatas.append(
        {
            "source": testset_record.metadata[0]["source"],
            "evolution_type": testset_record.evolution_type,
        }
    )

In [15]:
client.create_examples(
    inputs=inputs,
    outputs=outputs,
    metadata=metadatas,
    dataset_id=dataset.id,
)

## 7.5. LangSmith と Ragas を使ったオフライン評価の実装


### カスタム Evaluator の実装


In [16]:
from typing import Any

from langchain_core.embeddings import Embeddings
from langchain_core.language_models import BaseChatModel
from langsmith.schemas import Example, Run
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.llms import LangchainLLMWrapper
from ragas.metrics.base import Metric, MetricWithEmbeddings, MetricWithLLM


class RagasMetricEvaluator:
    def __init__(self, metric: Metric, llm: BaseChatModel, embeddings: Embeddings):
        self.metric = metric

        # LLMとEmbeddingsをMetricに設定
        if isinstance(self.metric, MetricWithLLM):
            self.metric.llm = LangchainLLMWrapper(llm)
        if isinstance(self.metric, MetricWithEmbeddings):
            self.metric.embeddings = LangchainEmbeddingsWrapper(embeddings)

    def evaluate(self, run: Run, example: Example) -> dict[str, Any]:
        context_strs = [doc.page_content for doc in run.outputs["contexts"]]

        # Ragasの評価メトリクスのscoreメソッドでスコアを算出
        score = self.metric.score(
            {
                "question": example.inputs["question"],
                "answer": run.outputs["answer"],
                "contexts": context_strs,
                "ground_truth": example.outputs["ground_truth"],
            },
        )
        return {"key": self.metric.name, "score": score}

In [17]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from ragas.metrics import answer_relevancy, context_precision

metrics = [context_precision, answer_relevancy]

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)  # きちんと評価したい場合は、gpt-4o, o1-mini, o1-preview を推奨
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

evaluators = [RagasMetricEvaluator(metric, llm, embeddings).evaluate for metric in metrics]

### 推論の関数の実装


In [18]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
db = Chroma.from_documents(documents, embeddings)

In [19]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''\
以下の文脈だけを踏まえて質問に回答してください。

文脈: """
{context}
"""

質問: {question}
''')

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

retriever = db.as_retriever()

chain = RunnableParallel(
    {
        "question": RunnablePassthrough(),
        "context": retriever,
    }
).assign(answer=prompt | model | StrOutputParser())

In [20]:
def predict(inputs: dict[str, Any]) -> dict[str, Any]:
    question = inputs["question"]
    output = chain.invoke(question)
    return {
        "contexts": output["context"],
        "answer": output["answer"],
    }

### オフライン評価の実装・実行


In [ ]:
from langsmith.evaluation import evaluate

evaluate(
    predict,
    data="agent-book",
    evaluators=evaluators,  # type: ignore
)

View the evaluation results for experiment: 'helpful-lunch-68' at:

https://smith.langchain.com/o/xxxxxxx

みたいに出力されます。


## LangSmith を使ったオンライン評価の実装


### フィードバックボタンを表示する関数の実装


In [22]:
from uuid import UUID

import ipywidgets as widgets
from IPython.display import display
from langsmith import Client


def display_feedback_buttons(run_id: UUID) -> None:
    # GoodボタンとBadボタンを準備
    good_button = widgets.Button(
        description="Good",
        button_style="success",
        icon="thumbs-up",
    )
    bad_button = widgets.Button(
        description="Bad",
        button_style="danger",
        icon="thumbs-down",
    )

    # クリックされた際に実行される関数を定義
    def on_button_clicked(button: widgets.Button) -> None:
        if button == good_button:
            score = 1
        elif button == bad_button:
            score = 0
        else:
            raise ValueError(f"Unknown button: {button}")

        client = Client()
        client.create_feedback(run_id=run_id, key="thumbs", score=score)
        print("フィードバックを送信しました")

    # ボタンがクリックされた際にon_button_clicked関数を実行
    good_button.on_click(on_button_clicked)
    bad_button.on_click(on_button_clicked)

    # ボタンを表示
    display(good_button, bad_button)

### フィードバックボタンを表示


In [23]:
from langchain_core.tracers.context import collect_runs

# LangSmithのトレースのID(Run ID)を取得するため、collect_runs関数を使用
with collect_runs() as runs_cb:
    output = chain.invoke("LangChainの概要を教えて")
    print(output["answer"])
    run_id = runs_cb.traced_runs[0].id

display_feedback_buttons(run_id)

LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。このフレームワークは、LLMアプリケーションのライフサイクルの各段階を簡素化します。具体的には、以下のような機能を提供しています。

1. **開発**: LangChainのオープンソースのビルディングブロックやコンポーネント、サードパーティの統合を使用してアプリケーションを構築できます。また、LangGraphを使用して、状態を持つエージェントを構築し、ストリーミングや人間の介入をサポートします。

2. **生産化**: LangSmithを使用して、チェーンを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。

3. **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換することができます。

LangChainは、以下のオープンソースライブラリで構成されています：
- `langchain-core`: 基本的な抽象化とLangChain表現言語。
- `langchain-community`: サードパーティの統合。
- `langchain`: アプリケーションの認知アーキテクチャを構成するチェーン、エージェント、検索戦略。
- LangGraph: LLMを使用して堅牢で状態を持つマルチアクターアプリケーションを構築するためのライブラリ。
- LangServe: LangChainチェーンをREST APIとしてデプロイするためのツール。
- LangSmith: LLMアプリケーションをデバッグ、テスト、評価、監視するための開発者プラットフォーム。

LangChainは、PythonとJavaScriptの両方のライブラリがあり、特にPythonのLangChainライブラリに焦点を当てたドキュメントが提供されています。


Button(button_style='success', description='Good', icon='thumbs-up', style=ButtonStyle())

Button(button_style='danger', description='Bad', icon='thumbs-down', style=ButtonStyle())

フィードバックを送信しました
フィードバックを送信しました
フィードバックを送信しました
フィードバックを送信しました
フィードバックを送信しました
フィードバックを送信しました
フィードバックを送信しました


全部実行し終わって、他の章に進む場合は、

1. chap07用の dependencies をコメントアウトして、
2. juv pyproject.toml で chap07以外用の dependencies をコメントインして、
3. `uv sync` するのを忘れないで下さい。
